This file will be used to contain data processing components

In [14]:
%pip install boto3 pyspark delta-spark python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [24]:
import os
from pyspark.sql import SparkSession

In [25]:
# You can use the following to set the environment variables in the notebook if you don't set manually access key, secret key and endpoint in minio
os.environ['OBJ_STORAGE_ACCESS_KEY'] = 'XpSkdzaL5NsSIe4YgHDg'
os.environ['OBJ_STORAGE_SECRET_KEY'] = 'jNiQdM05jn4cZIA6fBuWIjGD0UjfhsiKS3RpPL9W'
os.environ['OBJ_STORAGE_ENDPOINT'] = 'http://localhost:9000'

In [26]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')

In [27]:
path_1 = "s3a://data/data-raw/data.json"
path_2 = "s3a://data/data-raw/data2.json"
path_3 = "s3a://data/data-raw/data3.json"

In [43]:
# You need to more configuration if you want to use minio as object storage
# (hint: maybe you can using .config() method to set the configuration
# if you want using spark to read/write data from/to minio)

# Add the following configuration to read/write data from/to minio
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.hadoop.fs.s3a.endpoint", obj_storage_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", obj_storage_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", obj_storage_secret_key) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .getOrCreate()

# Read the JSON files
df1 = spark.read.json(path_1)
df2 = spark.read.json(path_2)
df3 = spark.read.json(path_3)

# Merge the DataFrames
df = df1.union(df2).union(df3)

# Write the result to a single JSON file
df.coalesce(1).write.json('result.json')

24/03/25 15:23:43 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://data/data-raw/data.json.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelat

Py4JJavaError: An error occurred while calling o139.json.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(D

The error message `java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found` indicates that the Hadoop-AWS library, which provides support for AWS S3, is not available in your classpath.

To resolve this issue, you need to include the Hadoop-AWS library and its dependencies in your PySpark environment. If you're using PySpark installed via pip, you can manually download the necessary JAR files and include them in your `spark.jars` configuration.

Here's an example of how to do this:

1. Download the following JAR files:

   - [hadoop-aws-3.2.0.jar](https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.2.0/hadoop-aws-3.2.0.jar)
   - [aws-java-sdk-bundle-1.11.375.jar](https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.11.375/aws-java-sdk-bundle-1.11.375.jar)

2. Save these JAR files in a directory accessible to your PySpark application.

3. Include these JAR files in your `spark.jars` configuration:



In [ ]:
spark = SparkSession.builder \
    .appName('app') \
    .config("spark.jars", "/path/to/hadoop-aws-3.2.0.jar,/path/to/aws-java-sdk-bundle-1.11.375.jar") \
    .config("spark.hadoop.fs.s3a.endpoint", obj_storage_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", obj_storage_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", obj_storage_secret_key) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .getOrCreate()



Replace `/path/to/hadoop-aws-3.2.0.jar` and `/path/to/aws-java-sdk-bundle-1.11.375.jar` with the actual paths where you saved the downloaded JAR files.

Please note that the versions of the JAR files should match the version of Hadoop that your Spark is using. The provided links are for Hadoop 3.2.0, which is used by Spark 3.0.0. If you're using a different version of Spark, you may need to adjust the versions of the JAR files accordingly.